# Easy Dev for Post-hoc OOD Detectors

This notebook integrates some simple post-hoc OOD detection methods.

We choose ImageNet-1K as in-distribution (ID) and load a pretrained vision transformer (ViT).

In [1]:
%load_ext autoreload
%autoreload 2

## Load Models and Dataset

In [2]:
from openood.utils import config
from openood.datasets import get_dataloader, get_ood_dataloader
from openood.evaluators import get_evaluator
from openood.networks import get_network
from torchvision.transforms import RandAugment
from torchvision import transforms

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/gridsan/nhulkund/.local/lib/python3.8/site-packages/mmcv/cnn/bricks/transformer.py:32: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


In [3]:
# # load config files for cifar100 baseline
# PATH="/home/gridsan/nhulkund/OpenOOD/"
# config_files = [
#     './configs/datasets/cifar100/cifar100.yml',
#     './configs/datasets/cifar100/cifar100_ood.yml',
#     './configs/networks/resnet18_32x32.yml',
#     './configs/pipelines/test/test_ood.yml',
#     './configs/preprocessors/base_preprocessor.yml',
#     './configs/postprocessors/msp.yml',
# ]
# config = config.Config(*config_files)
# # modify config 
# #config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
# config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/cifar100_res18_acc78.20.ckpt'
# config.network.pretrained = True
# config.num_workers = 8
# config.save_output = False
# config.parse_refs()

In [4]:
# load config files for cifar10 baseline
PATH="/home/gridsan/nhulkund/OpenOOD/"
config_files = [
    './configs/datasets/cifar10/cifar10.yml',
    './configs/datasets/cifar10/cifar10_ood.yml',
    './configs/networks/resnet18_32x32.yml',
    './configs/pipelines/test/test_ood.yml',
    './configs/preprocessors/base_preprocessor.yml',
    './configs/postprocessors/msp.yml',
]
config = config.Config(*config_files)
# modify config 
#config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/cifar10_res18_acc94.30.ckpt'
config.network.pretrained = True
config.num_workers = 8
config.save_output = False
config.parse_refs()


In [5]:
# # load config files for cifar10 baseline
# PATH="/home/gridsan/nhulkund/OpenOOD/"
# config_files = [
#     './configs/datasets/imagenet/imagenet.yml',
#     './configs/datasets/imagenet/imagenet_ood.yml',
#     './configs/networks/resnet50.yml',
#     './configs/pipelines/test/test_ood.yml',
#     './configs/preprocessors/base_preprocessor.yml',
#     './configs/postprocessors/msp.yml',
# ]
# config = config.Config(*config_files)
# # modify config 
# #config.network.checkpoint = PATH+'/results/cifar10_resnet18_32x32_base_e100_lr0.1/best.ckpt'
# config.network.checkpoint = PATH+'/scripts/download/results/checkpoints/imagenet_res50_acc76.10.pth'
# config.network.pretrained = True
# config.num_workers = 8
# config.save_output = False
# config.parse_refs()

In [6]:
config.network

checkpoint: /home/gridsan/nhulkund/OpenOOD//scripts/download/results/checkpoints/cifar10_res18_acc94.30.ckpt
name: resnet18_32x32
num_classes: 10
num_gpus: 1
pretrained: True

In [7]:
print(config)

dataset:
    image_size: 32
    interpolation: bilinear
    name: cifar10
    normalization_type: cifar10
    num_classes: 10
    num_gpus: 1
    num_machines: 1
    num_workers: 8
    pre_size: 32
    split_names: ['train', 'val', 'test']
    test:
        batch_size: 200
        data_dir: ./data/images_classic/
        dataset_class: ImglistDataset
        imglist_pth: ./data/benchmark_imglist/cifar10/test_cifar10.txt
        shuffle: False
    train:
        batch_size: 128
        data_dir: ./data/images_classic/
        dataset_class: ImglistDataset
        imglist_pth: ./data/benchmark_imglist/cifar10/train_cifar10.txt
        shuffle: True
    val:
        batch_size: 200
        data_dir: ./data/images_classic/
        dataset_class: ImglistDataset
        imglist_pth: ./data/benchmark_imglist/cifar10/val_cifar10.txt
        shuffle: False
evaluator:
    name: ood
exp_name: cifar10_resnet18_32x32_test_ood_ood_msp_default
machine_rank: 0
mark: default
merge_option: default
netwo

In [8]:
# get dataloader
id_loader_dict = get_dataloader(config)
ood_loader_dict = get_ood_dataloader(config)
# init network
net = get_network(config.network).cuda()
# init ood evaluator
evaluator = get_evaluator(config)

Model Loading resnet18_32x32 Completed!


In [9]:
# print(len(id_loader_dict['test'].dataset))
# print(len(id_loader_dict['val'].dataset))
# print(len(id_loader_dict['train'].dataset))
# print(len(ood_loader_dict['nearood']['tin'].dataset))

## Feature Extraction

In [10]:
from tqdm import tqdm
import numpy as np
import torch
import os.path as osp
import os
import cv2
import matplotlib.pyplot as plt
from skimage.exposure import rescale_intensity
from skimage import img_as_ubyte
from PIL import Image



In [11]:
def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def augment_preprocess(im):
    rand_aug = RandAugment(num_ops=2, magnitude=15)#num_ops=1, magnitude=5)
    #print(im[1][1][1])
    data_max, data_min = im.clone().max(), im.clone().min()
    pre_im = preprocess(im, data_max, data_min)
    #print(pre_im[1][1][1])
    #imshow(pre_im)
    aug_im = rand_aug(pre_im)
    post_im = postprocess(aug_im, data_max, data_min)
    #print(post_im[1][1][1])
    return post_im

def preprocess(data, data_max, data_min):
    data = (data - data_min) / (data_max - data_min) # normalize the data to 0 - 1
    #print("after normalise", data[1][1][1])
    data = 255 * data # Now scale by 255
    #print("after 255", data[1][1][1])
    data = data.type(torch.uint8)
    #print("after uint", data[1][1][1])
    return data

def postprocess(data, data_max, data_min):
    data = data.type(torch.float32)
    #print("after float", data[1][1][1])
    data /= 255
    #print("after 255 divide", data[1][1][1])
    data = (data * (data_max - data_min)) + data_min
    #print("after unnormalise", data[1][1][1])
    return data
    
    

# def preprocess(im):
#     im = rescale_intensity(im.numpy(), in_range='image', out_range=(0,1))
#     im = img_as_ubyte(im)
#     im = im.astype(np.uint8)
#     return im

In [12]:
from torchvision.datasets import CIFAR10, CIFAR100
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = CIFAR10(root='./data', train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
dataiter = iter(trainloader)
images, labels = next(dataiter)

In [13]:
# img=images[1]

# imshow(img)
# #print(img)
# #aug_img=augment_preprocess(img)
# rand_aug=RandAugmentManifold(2,9)
# aug_img=rand_aug(img)
# print("AT END")
# plt.imshow(np.transpose(aug_img, (1, 2, 0)), interpolation='nearest')

In [14]:
def save_arr_to_dir(arr, dir):
    os.makedirs(os.path.dirname(dir), exist_ok=True)
    with open(dir, 'wb+') as f:
        np.save(f, arr)

In [15]:
save_root = PATH+f'/results/{config.exp_name}'


In [16]:
RAND_N=3
RAND_M=5

In [17]:
# save id (test & val) results
net.eval()
modes = ['test', 'val']
for mode in modes:
    dl = id_loader_dict[mode]
    dataiter = iter(dl)
    
    logits_list = []
    feature_list = []
    label_list = []
    id_list = []
    count=0
    print(id_list)
    for i in tqdm(range(1,
                    len(dataiter) + 1),
                    desc='Extracting reults...',
                    position=0,
                    leave=True):
        batch = next(dataiter)
        data = batch['data'].cuda()
        label = batch['label']
        with torch.no_grad():
            logits_cls, feature = net(data, return_feature=True)
        logits_list.append(logits_cls.data.to('cpu').numpy())
        feature_list.append(feature.data.to('cpu').numpy())
        label_list.append(label.numpy())
    logits_arr = np.concatenate(logits_list)
    feature_arr = np.concatenate(feature_list)
    label_arr = np.concatenate(label_list)
    
    save_arr_to_dir(logits_arr, osp.join(save_root, 'id', f'{mode}_logits.npy'))
    save_arr_to_dir(feature_arr, osp.join(save_root, 'id', f'{mode}_feature.npy'))
    save_arr_to_dir(label_arr, osp.join(save_root, 'id', f'{mode}_labels.npy'))

[]


Extracting reults...: 100%|██████████| 45/45 [00:16<00:00,  2.71it/s]


[]


Extracting reults...: 100%|██████████| 5/5 [00:02<00:00,  2.02it/s]


In [18]:
# save ood results
net.eval()
ood_splits = ['nearood', 'farood']
for ood_split in ood_splits:
    for dataset_name, ood_dl in ood_loader_dict[ood_split].items():
        dataiter = iter(ood_dl)
    
        logits_list = []
        feature_list = []
        label_list = []

        for i in tqdm(range(1,
                        len(dataiter) + 1),
                        desc='Extracting reults...',
                        position=0,
                        leave=True):
            batch = next(dataiter)
            data = batch['data'].cuda()
            label = batch['label']
            aug_data=[]
            aug_label=[]
            for k in range(data.shape[0]):
                count+=1
                orig_img=data[k]
                aug_data.append(orig_img.cpu())
                aug_label.append(label[k])
                for j in range(10):
                    id_list.append(count)
                    aug_img=augment_preprocess(orig_img.cpu())
                    aug_img=torch.Tensor(aug_img)
                    aug_data.append(aug_img)
                    aug_label.append(label[k])
            aug_data=torch.stack(aug_data).cuda()
            aug_label=torch.stack(aug_label)
            with torch.no_grad():
                logits_cls, feature = net(aug_data, return_feature=True)
            logits_list.append(logits_cls.data.to('cpu').numpy())
            feature_list.append(feature.data.to('cpu').numpy())
            #label_list.append(aug_label.numpy())
            label_list.append(label.numpy())
        
        logits_arr = np.concatenate(logits_list)
        feature_arr = np.concatenate(feature_list)
        label_arr = np.concatenate(label_list)
        
        
        save_arr_to_dir(logits_arr, osp.join(save_root, ood_split, f'{dataset_name}_logits.npy'))
        save_arr_to_dir(feature_arr, osp.join(save_root, ood_split, f'{dataset_name}_feature.npy'))
        save_arr_to_dir(label_arr, osp.join(save_root, ood_split, f'{dataset_name}_labels.npy'))
        save_arr_to_dir(id_list, osp.join(save_root, ood_split, f'{dataset_name}_ids.npy'))

Extracting reults...: 100%|██████████| 275/275 [05:13<00:00,  1.14s/it]


In [19]:
#logits_arr.shape

## MSP Evaluation

In [20]:
# build msp method (pass in pre-saved logits)
def msp_postprocess(logits):
    score = torch.softmax(logits, dim=1)
    conf, pred = torch.max(score, dim=1)
    return pred, conf

# def msp_aug_postprocess(logits):
#     print(logits.shape, logits.shape[0]/11)
#     score = torch.softmax(logits, dim=1)
#     if score.shape[0] % 11 == 0: 
#         first_dim=int(score.shape[0]/11)
#         second_dim=11
#     else:
#         print(score.shape)
#     conf, pred = torch.max(score, dim=1)
#     pred_reshaped = pred.numpy().reshape(first_dim,second_dim)
#     pred_averaged = torch.Tensor(np.mean(pred_reshaped,axis=1))
#     conf_reshaped = conf.numpy().reshape(first_dim,second_dim)
#     conf_averaged = torch.Tensor(np.mean(conf_reshaped,axis=1))
#     return pred_averaged, conf_averaged

def msp_aug_postprocess(logits):
    score = torch.softmax(logits, dim=1)
    if score.shape[0] % 11 == 0: 
        first_dim=int(score.shape[0]/11)
        second_dim=11
    else:
        print(score.shape)
        print("not divisible")
    score_reshaped=score.numpy().reshape(first_dim,second_dim,10)
    score_averaged=torch.Tensor(np.mean(score_reshaped,axis=1))
    conf, pred = torch.max(score_averaged, dim=1)
    return pred, conf

In [21]:
# load logits, feature, label for this benchmark
results = dict()
# for id
modes = ['val', 'test']
results['id'] = dict()
for mode in modes:
    results['id'][mode] = dict()
    results['id'][mode]['feature'] = np.load(osp.join(save_root, 'id', f'{mode}_feature.npy'))
    results['id'][mode]['logits'] = np.load(osp.join(save_root, 'id', f'{mode}_logits.npy'))
    results['id'][mode]['labels'] = np.load(osp.join(save_root, 'id', f'{mode}_labels.npy'))

# for ood
split_types = ['nearood', 'farood']
for split_type in split_types:
    results[split_type] = dict()
    dataset_names = config['ood_dataset'][split_type].datasets
    for dataset_name in dataset_names:
        results[split_type][dataset_name] = dict()
        results[split_type][dataset_name]['feature'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_feature.npy'))
        results[split_type][dataset_name]['logits'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_logits.npy'))
        results[split_type][dataset_name]['labels'] = np.load(osp.join(save_root, split_type, f'{dataset_name}_labels.npy'))
        


In [22]:
# print(results['nearood']['cifar10']['logits'].shape)
# print(results['nearood']['cifar10']['labels'].shape)

In [23]:
def print_nested_dict(dict_obj, indent = 0):
    ''' Pretty Print nested dictionary with given indent level  
    '''
    # Iterate over all key-value pairs of dictionary
    for key, value in dict_obj.items():
        # If value is dict type, then print nested dict 
        if isinstance(value, dict):
            print(' ' * indent, key, ':', '{')
            print_nested_dict(value, indent + 2)
            print(' ' * indent, '}')
        else:
            print(' ' * indent, key, ':', value.shape)


In [24]:
print_nested_dict(results)

 id : {
   val : {
     feature : (1000, 512)
     logits : (1000, 10)
     labels : (1000,)
   }
   test : {
     feature : (9000, 512)
     logits : (9000, 10)
     labels : (9000,)
   }
 }
 nearood : {
   cifar100 : {
     feature : (99000, 512)
     logits : (99000, 10)
     labels : (9000,)
   }
   tin : {
     feature : (96723, 512)
     logits : (96723, 10)
     labels : (8793,)
   }
 }
 farood : {
   mnist : {
     feature : (770000, 512)
     logits : (770000, 10)
     labels : (70000,)
   }
   svhn : {
     feature : (286352, 512)
     logits : (286352, 10)
     labels : (26032,)
   }
   texture : {
     feature : (62040, 512)
     logits : (62040, 10)
     labels : (5640,)
   }
   place365 : {
     feature : (387145, 512)
     logits : (387145, 10)
     labels : (35195,)
   }
 }


In [25]:
# get pred, conf, gt from MSP postprocessor (can change to your custom_postprocessor here)
postprocess_results = dict()
# id
modes = ['val', 'test']
postprocess_results['id'] = dict()
for mode in modes:
    pred, conf = msp_postprocess(torch.from_numpy(results['id'][mode]['logits']))
    pred, conf = pred.numpy(), conf.numpy()
    gt = results['id'][mode]['labels']
    postprocess_results['id'][mode] = [pred, conf, gt]

# ood
split_types = ['nearood', 'farood']
for split_type in split_types:
    postprocess_results[split_type] = dict()
    dataset_names = config['ood_dataset'][split_type].datasets
    for dataset_name in dataset_names:
        pred, conf = msp_aug_postprocess(torch.from_numpy(results[split_type][dataset_name]['logits']))
        pred, conf = pred.numpy(), conf.numpy()
        gt = results[split_type][dataset_name]['labels']
        gt = -1 * np.ones_like(gt)   # hard set to -1 here
        postprocess_results[split_type][dataset_name] = [pred, conf, gt]

In [26]:
def print_all_metrics(metrics):
    [fpr, auroc, aupr_in, aupr_out,
        ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
        = metrics
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
            end=' ',
            flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
            flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
            end=' ',
            flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)   

In [27]:
from openood.evaluators.metrics import compute_all_metrics
import pandas as pd
def eval_ood(postprocess_results):
    [id_pred, id_conf, id_gt] = postprocess_results['id']['test']
    split_types = ['nearood', 'farood']

    for split_type in split_types:
        metrics_list = []
        print(f"Performing evaluation on {split_type} datasets...")
        dataset_names = config['ood_dataset'][split_type].datasets
        
        for dataset_name in dataset_names:
            [ood_pred, ood_conf, ood_gt] = postprocess_results[split_type][dataset_name]

            pred = np.concatenate([id_pred, ood_pred])
            conf = np.concatenate([id_conf, ood_conf])
            label = np.concatenate([id_gt, ood_gt])
            print(f'Computing metrics on {dataset_name} dataset...')
            
            ood_metrics = compute_all_metrics(conf, label, pred)
            df=pd.DataFrame(ood_metrics)
            df.to_csv('ood_metrics_trial')
            print_all_metrics(ood_metrics)
            metrics_list.append(ood_metrics)
        print('Computing mean metrics...', flush=True)
        metrics_list = np.array(metrics_list)
        metrics_mean = np.mean(metrics_list, axis=0)   
        print_all_metrics(metrics_mean)

 

In [28]:
eval_ood(postprocess_results)

Performing evaluation on nearood datasets...
Computing metrics on cifar100 dataset...
FPR@95: 14.17, AUROC: 97.45 AUPR_IN: 98.01, AUPR_OUT: 96.70
CCR: 15.41, 58.57, 82.67, 91.74, ACC: 95.24
──────────────────────────────────────────────────────────────────────
Computing metrics on tin dataset...
FPR@95: 11.95, AUROC: 97.70 AUPR_IN: 98.23, AUPR_OUT: 96.99
CCR: 11.18, 56.36, 85.21, 92.01, ACC: 95.24
──────────────────────────────────────────────────────────────────────
Computing mean metrics...
FPR@95: 13.06, AUROC: 97.57 AUPR_IN: 98.12, AUPR_OUT: 96.85
CCR: 13.29, 57.46, 83.94, 91.88, ACC: 95.24
──────────────────────────────────────────────────────────────────────
Performing evaluation on farood datasets...
Computing metrics on mnist dataset...
FPR@95: 4.45, AUROC: 98.46 AUPR_IN: 96.26, AUPR_OUT: 99.73
CCR: 67.66, 83.98, 90.91, 93.04, ACC: 95.24
──────────────────────────────────────────────────────────────────────
Computing metrics on svhn dataset...
FPR@95: 4.48, AUROC: 98.73 AUPR_IN